In [1]:
import pandas as pd
import numpy as np
import os.path as ospath

In [9]:
excelfile = ospath.join('..','Serious games','Games','БД4_Лёгкая.xlsm')

Alldata = pd.read_excel(excelfile,sheet_name='Данные')
Games = pd.read_excel(excelfile,sheet_name='Игры')
print(Alldata.columns)
Games.columns

In [11]:
YHGames=Games[(Games.Mechname=='yh') & (Games.GameMech=='agree')]
YHExpGames=Games[(Games.Mechname=='yh') & (Games.GameMech=='exp')]

YHData=Alldata[Alldata.Game.isin(YHGames.Game)]
YHEData=Alldata[Alldata.Game.isin(YHExpGames.Game)]

In [13]:
print(YHData.shape,YHEData.shape)

(1035, 16) (75, 16)


In [85]:
GLGames=Games[(Games.Mechname=='gl') & (Games.GameMech=='agree')]
GLData=Alldata[Alldata.Game.isin(GLGames.Game)]

GLEGames=Games[(Games.Mechname=='gl') & (Games.GameMech=='exp')]
GLEData=Alldata[Alldata.Game.isin(GLEGames.Game)]

In [86]:
GLData.shape

(2847, 16)

## Нэш-торги

In [20]:
YHg=YHData.groupby(['Game','Time'])

In [53]:
YHg.sum();

### Функции расчёта выигрышей в игре

In [14]:
class Game:
    import numpy as np
    def __init__(self,R,s0,types):
        self.R = R
        self.s0 = np.array(s0)
        self.n = len(s0)
        self.types = np.array(types)
    def u(self,x):
        return self.np.sqrt(self.types+x)
game = Game( 115,(115/3,115/3,115/3), (1,9,25))

class Mechanism:
    def __init__(self,game,params,xfunc,tfunc):
        self.game = game
        for k,v in params.items():
            setattr(self,k,v)
        self.xfunc = xfunc
        self.tfunc = tfunc
        
    def x(self,s):
        return self.xfunc(s,self.game)
    def t(self,s):
        return self.tfunc(s,self.game,self)
    def f(self,s):
        return game.u(self.x(s))-self.t(s)

YHMechanism = Mechanism(game,
                        {'beta':0.0005}, 
                        lambda s,g: s*g.R/s.sum(),
                        lambda s,g,m: m.beta*s*(np.repeat(s.sum(),3)-s))

class GLClass:
    from scipy.spatial import distance
    def __init__(self):
        self.glx = lambda s,g: s.sum(axis=0) / g.n
    def glt(self,s,g,m):
        p = m.beta * self.distance.cdist([self.glx(s,g)],s,'sqeuclidean')
        pmean = m.alfa * p.mean()
        return p - pmean
GL = GLClass()
    
GLMechanism = Mechanism(game,{'beta':0.0005,'alfa':1},
                        GL.glx,
                        GL.glt
                       )

In [17]:
print(YHMechanism.beta)
s = np.array([1.2,40,80])
print(YHMechanism.t(s))
YHMechanism.f(s)

0.0005
[0.072 1.624 1.648]


array([1.39040003, 5.22828395, 8.39727704])

In [18]:
def Unash(f,f0):
    df = f-f0
    return (np.fabs(df)).prod() * np.min(np.sign(df))

print(YHMechanism.game.s0)

f0 = YHMechanism.game.u([0,0,0])
print(f0)

Unash(f0,s)

[38.33333333 38.33333333 38.33333333]
[1. 3. 5.]


-554.9999999999999

### Цикл получения исходных данных для Нэш-торгов

In [35]:
import itertools
res = []
for name,group in YHg:
    group = group.sort_values(by='GrSubject')
    if name[1] == 1 :
        curname = name[0]
        prevg = group
        continue
    
    fprev = prevg['Gain'].values
    sprev = prevg['s2'].values
    s = group['s2'].values
    s1, s2, s3 = sprev.copy(), sprev.copy(), sprev.copy()
    s1[0] = s[0]
    s2[1] = s[1]
    s3[2] = s[2]
    f1 = YHMechanism.f(s1)
    f2 = YHMechanism.f(s2)
    f3 = YHMechanism.f(s3)
    U = Unash(fprev,f0)
    U1 = Unash(f1,f0)
    U2 = Unash(f2,f0)
    U3 = Unash(f3,f0)
    Unew = Unash(YHMechanism.f(s),f0)
    res.append( [a for a in itertools.chain(name,[U1>U,U2>U,U3>U,Unew>U,U,U1,U2,U3,Unew],s)] )
    prevg = group
    #print(name,prev['Gain'])

In [52]:
a = np.vstack(res)
data_a = pd.DataFrame(a); data_a;

In [45]:
pda = data_a[[2,3,4,5]]
pdaPar = data_a[data_a[6]>0][[2,3,4,5]]

## YH

Игры с механизмом YH и механизмом остановки "по договорённости" - никто не поменял заявок.
Всего шагов 1035, если брать для каждого игрока в отдельности, то 3105 = 1035 * 3

Функция Нэша: $ U(f,f^0)=\prod_{i} (f_i -f_i^0)') $

Базовые выигрыши: $ f^0 = u(0,0,0) $

Определения для текущего шага $t>1$: $$ U_{prev} = U(f(s^{t-1}),f^0)') $$
$$ U_i =U(f(s_i^t,s_{-i}^{t-1}),f^0) $$
$$ U_{new}=U(f(s^t),f^0) $$

In [36]:
def Nash_stat(data):
    a = (data[data>0].count()).values
    return [a[0],a[1],a[2],data[(data[2]*data[3]*data[4]==1)].shape[0],
                                       data[data[2]*data[3]*data[4]*data[5]==1].shape[0],
                                       data[data[5]==1].shape[0],data[(data[5]==1) & (data[2]*data[3]*data[4]==0)].shape[0],
                                       data[(data[5]==1) & (data[2]+data[3]+data[4]==0)].shape[0]
                                       ]

In [37]:
pd.DataFrame({'Критерии':['U1 > Uprev','U2 > Uprev','U3 > Uprev','U1>Uprev & U2>Uprev & U3>Uprev',
                          'U1>Uprev & U2>Uprev & U3>Uprev & Unew > Uprev','Unew > Uprev',
                          'Unew > Uprev & Exist Ui < Uprev','Unew > Uprev & All Ui < Uprev'],
              'Число подходящих ходов':Nash_stat(pda),
              'Число подх. ходов Uprev>0':Nash_stat(pdaPar)
             })

,Критерии,Число подходящих ходов,Число подх. ходов Uprev>0
0,U1 > Uprev,172,148
1,U2 > Uprev,181,155
2,U3 > Uprev,189,162
3,U1>Uprev & U2>Uprev & U3>Uprev,64,58
4,U1>Uprev & U2>Uprev & U3>Uprev & Unew > Uprev,64,58
5,Unew > Uprev,167,127
6,Unew > Uprev & Exist Ui < Uprev,103,69
7,Unew > Uprev & All Ui < Uprev,1,0


In [26]:
datafile = ospath.join('..','Serious games','data.csv')

brdata = pd.read_csv(datafile,header=None)

brdata.columns=['game','time','g1','g2','g3','br1','br2','br3']

jdata=pd.concat([data_a, brdata], axis=1, join='inner')

In [49]:
def BR_stat(data):
    return [data[(data['g1']>0)&(data['g1']<=1)].shape[0]+
            data[(data['g2']>0)&(data['g2']<=1)].shape[0]+
            data[(data['g3']>0)&(data['g3']<=1)].shape[0],
            data[(data['g1']>0)].shape[0]+
            data[(data['g2']>0)].shape[0]+
            data[(data['g3']>0)].shape[0],
            data[data['g1']==1].shape[0]+
            data[data['g2']==1].shape[0]+
            data[data['g3']==1].shape[0],
            data[(data['g1']>=0.9)&(data['g1']<=1.1)].shape[0]+
            data[(data['g2']>=0.9)&(data['g2']<=1.1)].shape[0]+
            data[(data['g3']>=0.9)&(data['g3']<=1.1)].shape[0],
            data[(data['g1']>=0.8)&(data['g1']<=1.2)].shape[0]+
            data[(data['g2']>=0.8)&(data['g2']<=1.2)].shape[0]+
            data[(data['g3']>=0.8)&(data['g3']<=1.2)].shape[0],
                                       ]

In [51]:
pd.DataFrame({'Критерии':['IB','IB+','BR(0)','BR(0.1)','BR(0.2)'],
              'Число подходящих ходов':BR_stat(jdata),
             })

,Критерии,Число подходящих ходов
0,IB,296
1,IB+,442
2,BR(0),0
3,BR(0.1),36
4,BR(0.2),59
